In [1]:
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, SimpleRNN, LSTM, GRU, Dense

In [2]:
num_words= 10000
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=num_words)

In [3]:
maxlen = 100
x_train = pad_sequences(x_train, maxlen=maxlen, padding='post', truncating='post')
x_test = pad_sequences(x_test, maxlen=maxlen, padding='post', truncating='post')

In [4]:
embedding =100
model = Sequential([
    Embedding(input_dim=num_words, output_dim=embedding, input_length=maxlen),
    Bidirectional(LSTM(5)),
    Dense(1, activation='sigmoid')
])
model.build()
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(x_train, y_train,
                    batch_size=num_words,
                    epochs=1)
model.summary()


c:\Users\HP\Desktop\VSCode\Bidirection_lstm\venv\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


3/3 ━━━━━━━━━━━━━━━━━━━━ 8s 995ms/step - accuracy: 0.5218 - loss: 0.6922


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 100, 100)       │     1,000,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 10)             │         4,240 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │            11 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,012,755 (11.49 MB)

 Trainable params: 1,004,251 (3.83 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,008,504 (7.66 MB)

In [5]:
train_loss, train_acc = model.evaluate(x_test, y_test, verbose=2)

782/782 - 8s - 10ms/step - accuracy: 0.5602 - loss: 0.6902


In [6]:
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

In [7]:
log_dir = "logs/fit" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

In [8]:
log_dir

'logs/fit20240723-195550'

In [9]:
model_callbacks = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [10]:
import mlflow

In [13]:
history = model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=100, callbacks=[model_callbacks, earLy_stop]
)

Epoch 1/100
782/782 ━━━━━━━━━━━━━━━━━━━━ 49s 62ms/step - accuracy: 0.7107 - loss: 0.5563 - val_accuracy: 0.8020 - val_loss: 0.4399
Epoch 2/100
782/782 ━━━━━━━━━━━━━━━━━━━━ 50s 64ms/step - accuracy: 0.8773 - loss: 0.3147 - val_accuracy: 0.7966 - val_loss: 0.4608
Epoch 3/100
782/782 ━━━━━━━━━━━━━━━━━━━━ 54s 69ms/step - accuracy: 0.9164 - loss: 0.2236 - val_accuracy: 0.8023 - val_loss: 0.5188
Epoch 4/100
782/782 ━━━━━━━━━━━━━━━━━━━━ 43s 55ms/step - accuracy: 0.9474 - loss: 0.1571 - val_accuracy: 0.7992 - val_loss: 0.5226
Epoch 5/100
782/782 ━━━━━━━━━━━━━━━━━━━━ 43s 55ms/step - accuracy: 0.9626 - loss: 0.1129 - val_accuracy: 0.7938 - val_loss: 0.6310
Epoch 6/100
782/782 ━━━━━━━━━━━━━━━━━━━━ 45s 58ms/step - accuracy: 0.9748 - loss: 0.0796 - val_accuracy: 0.7911 - val_loss: 0.7118
Epoch 7/100
782/782 ━━━━━━━━━━━━━━━━━━━━ 43s 55ms/step - accuracy: 0.9821 - loss: 0.0573 - val_accuracy: 0.7888 - val_loss: 0.8125
Epoch 8/100
782/782 ━━━━━━━━━━━━━━━━━━━━ 48s 61ms/step - accuracy: 0.9889 - loss: 0

In [14]:
earLy_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [15]:
from mlflow.models.signature import infer_signature
model_sign = infer_signature(x_test, model.predict(x_test))

782/782 ━━━━━━━━━━━━━━━━━━━━ 8s 10ms/step


In [16]:
exp_timestamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
exp_name= "IMDB_bidirection_lstm_time:" + exp_timestamp
#exp_name = "IMDB_bidirection_lstm"
run_timestamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
run_name = "IMDB_bidirection_lstm_run:" + run_timestamp

In [17]:
def tf_experiment_tracking(exp_name, run_name, train_loss, train_acc, model_sign):
    mlflow.set_experiment(exp_name)
    with mlflow.start_run(run_name=run_name):
        mlflow.log_param("batch_size", num_words )
        mlflow.log_param("learning_rate", 0.001)
        mlflow.log_param("epochs", 100)
        mlflow.log_metric("train_loss", train_loss)
        mlflow.log_metric("train_accuracy", train_acc)
        mlflow.tensorflow.log_model(model, "IMDB", signature=model_sign)
    mlflow.end_run()

In [18]:
tf_experiment_tracking(exp_name, run_name, train_loss, train_acc, model_sign)

2024/07/23 20:05:29 INFO mlflow.tracking.fluent: Experiment with name 'IMDB_bidirection_lstm_time:20240723-200529' does not exist. Creating a new experiment.


In [19]:
model.save("models/BI_LSTM_model.h5")